The purpose of this file is to perform experiments and keep notes. I prefer to do it this way to keep the other files cleaner and minimize necessary cleanup work later.

In [1]:
from collections import defaultdict
from pprint import pprint

In [2]:
import pandas as pd
from pymongo import MongoClient

client = MongoClient()
db = client.osm
bergen = db.bergen

# df = pd.DataFrame(list(db.bergen.find()))

In [3]:
#Creating function to list documents with a given feature

def docs_with_feature(feature,max_docs=5,value=None):
    
    if value:
        query_cursor = bergen.find( { feature: value } )
    else:
        query_cursor = bergen.find( { feature: {'$exists': True} } )
    cursor_count = query_cursor.count()
    
    print(cursor_count," documents have feature.")
    
    if cursor_count < max_docs+1:
        for doc in query_cursor:
            pprint(doc)
            print('----')
            
    else:
        count = 0
        print("Printing first {0} matching documents.".format(max_docs) )
        
        for doc in query_cursor:
            count += 1
            if count < max_docs:
                pprint(doc)
                print('----')
    

In [4]:
docs_with_feature('amenity')

3423  documents have feature.
Printing first 5 matching documents.
{'_id': ObjectId('58ee99f28a51df7ed61f5595'),
 'address': {'city': 'Frekhaug',
             'housenumber': '3',
             'postcode': '5918',
             'street': 'Holtermandsvegen'},
 'amenity': 'school',
 'created': {'changeset': '25431710',
             'timestamp': '2014-09-14T13:03:49Z',
             'uid': '88164',
             'user': 'Med',
             'version': '4'},
 'id': '659034',
 'name': 'Nordhordaland folkehøgskule',
 'operator': 'Nordhordland Indremisjon',
 'pos': [60.514765, 5.2414095],
 'type': 'node',
 'website': 'http://www.skolen.info/',
 'wikipedia': 'no:Nordhordland Folkehøgskule'}
----
{'_id': ObjectId('58ee99f28a51df7ed61f5bae'),
 'access': 'permissive',
 'amenity': 'parking_entrance',
 'created': {'changeset': '37097944',
             'timestamp': '2016-02-09T10:20:29Z',
             'uid': '1965308',
             'user': 'FredrikLindseth',
             'version': '11'},
 'foot': 'permis

In [5]:
feature_list = list(db.bergen.find())

In [6]:
feature_list[1000:1001]

[{'_id': ObjectId('58ee99f28a51df7ed61f55fb'),
  'created': {'changeset': '34180559',
   'timestamp': '2015-09-22T11:02:06Z',
   'uid': '3252623',
   'user': 'Alex_K_B',
   'version': '8'},
  'highway': 'traffic_signals',
  'id': '674462',
  'pos': [60.3924968, 5.3184896],
  'type': 'node'}]

In [7]:
df['created'][1:10]

NameError: name 'df' is not defined

In [ ]:
df['address'][1:10]

In [ ]:
tmp_find_one= bergen.find_one()

for col in tmp_find_one: 
    print(col)

In [ ]:
from IPython.display import display
from IPython.display import HTML

#Getting count of documents with address field

address_query = { 'address' : {'$exists' : True } }
address_documents = db.bergen.find(address_query)
address_count = address_documents.count()

address_count

In [ ]:
address_documents = db.bergen.find(address_query)#, { 
#         'address.street': 1,'address.housenumber': 1,'address.postcode': 1,'address.city': 1
#     }
#                                   )

In [ ]:
tmp_list = list(address_documents)

In [ ]:
len(tmp_list)

In [ ]:
tmp_list[1]

In [ ]:
from pprint import pprint

In [ ]:


to_df = list()
# columns = ['street','housenumber','postcode','city']
count = 0 

for doc in tmp_list:
    record = { 'id': doc['id'], 'type': doc['type'] }
    
    for key,val in doc['address'].items():
        record[key] = val
        
    for key,val in doc['created'].items():
        record['created_' + key] = val
    
    if 'pos' in doc.keys():
        record['lat'] = doc['pos'][0]
        record['lon'] = doc['pos'][1]
    
    to_df.append(record)

In [ ]:
tmp_list[1]

In [ ]:
from collections import defaultdict
# tmp_set = set()
tmp_def_dict = defaultdict(int)
for doc in tmp_list:
    
    for k,v in doc.items():
        
#         if k in ('address','created'):
        try:
            for key,val in v.items():
#             tmp_set.add(k+'_'+key)
                tmp_def_dict[k+':'+key] += 1
#         else:
        except AttributeError:
            tmp_def_dict[k] += 1
#             tmp_set.add(k)
    

In [ ]:
tmp_def_dict

In [ ]:
tmp_def_dict.keys()

In [ ]:
for doc in bergen.find( {'name': {'$exists': True}}):
    pprint(doc)

In [ ]:
tmp_s = pd.Series(tmp_def_dict)

tmp_s.sort_values(ascending=False).to_frame('count')

In [ ]:
pd.DataFrame(tmp_def_dict,index=range(len(tmp_def_dict.keys())) )

In [ ]:
tmp_set

In [ ]:
df_address = pd.DataFrame(to_df, columns = [
        'id','street','housenumber','city','postcode','created_uid','created_user',
        'created_version','created_timestamp','created_uid',
    'lat','lon','type'] )

In [ ]:
df_address[1:10]

In [ ]:
df_address.count()

In [ ]:
values = []
for cc in db.bergen.find():
  values.append(cc['created'])

print(pd.DataFrame(values)[1:10])

In [ ]:
import pandas as pd

postcodes_per_municipality = pd.read_csv('data/Postnummerregister_ansi.tsv', encoding='utf-8',delimiter='\t',header=0, names=[
        'postal_code','postal_place','muni_number','muni_name','category'],
            dtype = {'postal_code': str, 'municipality_number': str})

postcodes_per_municipality.head(5)

In [ ]:
postcodes_per_municipality['postcodes_per_municipality'] = pd.to_numeric(postcodes_per_municipality['postal_code'])

In [ ]:
import timeit
tmp_sl = postcodes_per_municipality['postal_code']


start = timeit.timeit()
for i in tmp_sl:
    if i == 5151:
        print('series for and if equal')

end = timeit.timeit()
print(end - start)

start = timeit.timeit()

if 5140 in tmp_sl.values:
        print('series if in')
    
end = timeit.timeit()
print(end - start)

tmp_sl = list(tmp_sl)

start = timeit.timeit()
for i in tmp_sl:
    if i == 5151:
        print('list for and if equal')

end = timeit.timeit()
print(end - start)

start = timeit.timeit()

if 5140 in tmp_sl:
        print('list if in')
    
end = timeit.timeit()
print(end - start)

# for func in f:
#     f = ['value = [val[5] for col,val in dictionary.items()]', 'value = s.loc[5]', 'value = df.iloc[5]']

#     print(min(timeit.Timer(func, setup).repeat(3, 100000)))

In [ ]:
%alias_magic t timeit

import timeit
tmp_sl = postcodes_per_municipality['postal_code']


for i in tmp_sl:
    if i == 5151:
        print('series for and if equal')

%t -n1 if str(5146) in tmp_sl.values: print('series if in')

tmp_sl = list(tmp_sl)

for i in tmp_sl: 
    if i == 5151: 
        print('list for and if equal')

%t -n1 if str(5146) in tmp_sl: print('list if in')

# for func in f:
#     f = ['value = [val[5] for col,val in dictionary.items()]', 'value = s.loc[5]', 'value = df.iloc[5]']

#     print(min(timeit.Timer(func, setup).repeat(3, 100000)))

In [ ]:
tmp_sl = postcodes_per_municipality['postal_code']


tmp_sl.values

In [ ]:
#Removed cell February 9, 2017

# aggregated = bergen.aggregate([  
#         {'$match' :  { 'address': {'$exists' : True }, 'type': 'way' } },
#         { "$group" : { 
#                 "_id" : "$id","count" : { "$sum" : 1} } }
#     ])

# unique_street_count = 0

# for doc in aggregated:
# #     household_count += doc['count']
#     unique_street_count += 1

# print(unique_street_count)

#Removed cell, February 9, 2017

#EXPERIMENTAL, WILL BE REMOVED
streetnames_sorted_dict = dict(sorted(addresses_on_street.items(), key=lambda x: x[1], reverse=True))

for rec in streetnames_sorted_dict:
    if streetnames_sorted_dict[rec] == 184:
        print (rec,streetnames_sorted_dict[rec],type(rec))

#Removed cell February 9, 2017

#EXPERIMENTAL, WILL BE REMOVED
tmp_agg = db.bergen.aggregate([
         { "$group" : { "_id" : "$address.street",
                       "mset" : {
                         "$addToSet" : "$address.house_number"
                 } } },
         { "$unwind" : "$mset"},
         { "$group" : { "_id" : "$_id",
                       "count" : { "$sum" : 1} } },
         { "$sort" : { "count" : -1 } },
         { "$limit" : 10 } ] )

for doc in tmp_agg:
    print(doc)

#Removed cell February 20, 2017

#Converting list for dataframe usage
to_df = []

for dic in duplicate_addresses:
    dictionary = dict()
    
#     for key,val in dic.items():
#         dictionary[key] = val
        
#         if key == '_id':
    for key,val in dic['_id'].items():
        dictionary[key] = val
        
    dictionary['count'] = dic['count']
                
        
    to_df.append(dictionary)
    
to_df

#Removed from cleaning part February 20, 2017

            if count < 10:
                pass
            elif count == 10:
                pprint(data)
            else:
                break
        
        addr_count = 0
        for node in data:
            if (addr_count < 20) and 'address' in node.keys():-=0
                addr_count += 1
                pprint(node['address'])  

In [ ]:
from collections import defaultdict

user_count_query = bergen.aggregate( [
   {
     '$group': {
        '_id' : { 'uid': '$created.uid', 'username': '$created.user' }
           }
        },
   {
     '$group': {
        '_id': 'null',
        'count': { '$sum': 1 }
     }
   }
] )

for doc in user_count_query:
    user_count = doc['count']

average_contributions = bergen.aggregate( [
   {
          '$group': 
            {
                '_id' : 
                { 'uid': '$created.uid', 'username': '$created.user' },
                'count': { '$sum': 1 } 
            } 
    },
    { 
            '$group': 
            {
                '_id': 'null',
                'avg': { '$avg': '$count' } 
            }
    }
] )

for doc in average_contributions:
    user_average = round(doc['avg'],2)
    
grouped_users = list(bergen.aggregate([  
        { 
            "$group" : 
            { 
                "_id" : { "uid": "$created.uid", "username": "$created.user" },
                "count" : { "$sum" : 1} 
            } 
        },
        { "$sort" : { "count" : 1 } }
        ]))

user_no = 0
halfway = round(user_count / 2)
mode_dict = defaultdict(int)

for doc in grouped_users:
        user_no += 1
        val = doc['count']
        if user_no == halfway:
            user_median = val
        
        mode_dict[val] += 1

user_mode = max(mode_dict.items(), key=lambda a: a[1])
mode_percentage = round((user_mode[1] / user_count) * 100,2)
            
print("Total user count:",user_count)
print("Average contributions per user:",user_average)
print("Median contributions per user:",user_median)
print("Mode of contribution count: {0} contributors ({1}%) submitted {2} edit.".format(
    user_mode[1],mode_percentage,user_mode[0] ) )



In [ ]:
import numpy as np
from matplotlib import pyplot as plt
% matplotlib inline


mu, sigma = 100, 15
x = mu + sigma*np.random.randn(10000)

bah = np.array(list(doc['count'] for doc in grouped_users))

print(len(x))

plt.hist(bah,bins=100)


In [ ]:
s_mode_dict = pd.Series(mode_dict)

In [ ]:
df_mode_dict = pd.DataFrame(s_mode_dict,columns=['count'])

In [ ]:
df_mode_dict['contributions'] = df_mode_dict.index

In [ ]:
df_mode_dict = df_mode_dict[['contributions','count']]

In [ ]:
[2,3,4,list(range(1,10,2))]

In [ ]:
list(range(1,10,2)) + list(range(1,4))

from collections import OrderedDict

bracket_names = OrderedDict()
for item in bins:
    if item != bins[-1]:
        if item < 10:
            bracket_names[str(item)] = item  
        else:
            next = bins[bins.index(item) + 1]
            bracket_names['{0} to {1}'.format(item,next)] = item
            
bracket_names

In [ ]:
bracket_names.keys()

In [ ]:
# for key,val in (bracket_names.keys(),bracket_names.values()):
#     print(key,val)

len(bracket_names.values())

print(bins)
print(list(bracket_names.values()))

tmp_dict = dict()

for item in bins:
    bin_index = bins.index(item)
        
    try:
        tmp_dict[bracket_names[bin_index]] = item
    except IndexError:
        print(item)
    
tmp_dict

In [ ]:
df_mode_dict.dtypes

In [ ]:
str(11000)[:-3] + 'K'

In [ ]:
#Creating age brackets and removing records with missing ageb

bins = list(range(0,10,1) ) + list(range(10,100,10) ) + \
list(range(100,1000,100) ) + list(range(1000,10000,5000) ) + \
list(range(10000,50000,20000) ) + list(range(50000,151000,50000) )

bracket_names = list()
for item in bins:
    if item != bins[-1]:
        if item < 10:
            bracket_names.append(str(item+1)) 
        else:
            start = item + 1
            next = bins[bins.index(item) + 1]
            if start > 1000:
                start = str(start)[:-3] + 'K'
                next = str(next)[:-3] + 'K'
            bracket_names.append('{0} to {1}'.format(start,next))
        
categories = pd.Series(pd.cut(df_mode_dict['contributions'], bins ,
                              labels=bracket_names, include_lowest=True ) )

df_mode_dict['bracket'] = categories
#pd.cut does not work perfectly for what I want, so I have to adjust some brackets
# df_mode_dict['bracket'][df_mode_dict['contributions'] < 11] = df_mode_dict['contributions']

display(df_mode_dict.head(30))
display(df_mode_dict.tail(4))

# print("Number of passengers used for age analysis:", len(age_data))

In [ ]:
for_plot = df_mode_dict.groupby('bracket')['count'].agg('sum')

In [ ]:
for_plot

In [ ]:
import seaborn as sns

fig = plt.figure(figsize=(8,5))
ax = plt.subplot(1,1,1)

#all passengers plot
rects = for_plot.plot(kind='bar',ax=ax,color='g')
ax.set_title('Contributions per user',fontsize='large',fontweight='bold')
ax.set_xlabel('Number of contributions',fontsize='small')
ax.set_ylabel('User count',fontsize='small')
ax.set_xticklabels(for_plot.index.values,rotation='70',fontsize='x-small')

plt.show()

In [ ]:
s_mode_dict.plot.hist(bins=10)

In [ ]:
bah_list = list(bah)

bah_list.sort()

bah_list[-1]

In [ ]:
s_grouped_users = pd.Series(list(doc['count'] for doc in grouped_users))

s_grouped_users.plot(kind='hist')

data = s_grouped_users

plt.hist(data, bins=range(min(data), max(data) + 5, 10))

In [ ]:
s_grouped_users.hist(bins=[0, 10, 20, 30, 40, 50, 100])

In [ ]:
df_mode_dict.iloc[50:]

In [ ]:
from scipy import stats

stats.describe(bah)

#### Misspelled streets

In [ ]:
#Getting counts for streetnames and addresses

aggregated = bergen.aggregate([  
        {'$match' : {'address': {'$exists' : True } } },
        { "$group" : { 
                "_id" : "$address.street","count" : { "$sum" : 1} } }
    ])

household_count = 0
unique_street_count = 0
addresses_on_street = {}

for doc in aggregated:
    household_count += doc['count']
    unique_street_count += 1
    
    addresses_on_street[doc['_id']] = doc['count']

print("total addresses in Bergen:", household_count)
print("number of streetnames:", unique_street_count)

In [ ]:
#Taking a look at the streets with the most addresses

from operator import itemgetter

streetnames_sorted_dict = dict(sorted(addresses_on_street.items(), key=lambda x: x[1], reverse=True)[:10])
streetnames_sorted_list = sorted(addresses_on_street.items(), key=lambda x: x[1], reverse=True)


display(HTML("<b>Streets with most addresses on them:</b>"))

for street,count in streetnames_sorted_list[0:10]:
    print(street,count)

In [ ]:
#Checking for potential duplicate data due to misspelled street names

import difflib
from fuzzywuzzy import fuzz

def fuzzy_streets(ratio,house_count):
    
    fuzzy_matches = list()
    compare_count = 0
    
    for k1 in streetnames_sorted_list:

        if k1[0] is None:
            print("Addresses without street name:",k1[1])

        #Only comparing street names with less addresses than house_count
        elif k1[1] <= house_count:
            
            compare_count += 1

            for k2 in streetnames_sorted_list:

                if k2[0] is None:
                    pass

                elif k2[0] == k1[0]:
                    pass

                else:                    
                    
                    fuzz_ratio = fuzz.ratio(k1[0],k2[0])
                    
                    if fuzz_ratio >= ratio:
                        fuzzy_matches.append({k1: k2,"fuzz ratio": fuzz_ratio})

    print("Number of street names compared: {0} of {1}".format(compare_count,len(streetnames_sorted_list)))
    
    return fuzzy_matches

In [ ]:
potential_misspellings = fuzzy_streets(92,10)

In [ ]:
#Printing out the potential misspellings

df_potential_misspellings = pd.DataFrame(columns = [
        'high_spelling','high_count','low_spelling','low_count','fuzz_ratio'])

#Adding index to make it easier to sort out the items I need to investigate further
count = 0

for spellings in potential_misspellings:
    count += 1
    df_potential_misspellings.loc[count] = None
    for key, val in spellings.items():
        if type(key) == tuple:
            if key[1] > val[1]:
                df_potential_misspellings.loc[count]['high_spelling'] = key[0]
                df_potential_misspellings.loc[count]['low_spelling'] = val[0]
                df_potential_misspellings.loc[count]['high_count'] = key[1]
                df_potential_misspellings.loc[count]['low_count'] = val[1]

            else:
                df_potential_misspellings.loc[count]['high_spelling'] = val[0]
                df_potential_misspellings.loc[count]['low_spelling'] = key[0]
                df_potential_misspellings.loc[count]['high_count'] = val[1]
                df_potential_misspellings.loc[count]['low_count'] = key[1]
        else:
            df_potential_misspellings.loc[count]['fuzz_ratio'] = val

df_potential_misspellings.drop_duplicates().sort_values('high_spelling',ascending=True)

Contributors

In [ ]:
#Getting counts for streetnames and addresses

user_count = bergen.aggregate([  
        { "$group" : { 
                "_id" : { "uid": "$created.uid", "mset": {"username": "$created.user" } } } },
        { "$unwind" : "$mset"},
        { "$group" : { "_id" : "$uid",
                      "count" : { "$sum" : 1} } }        
    ])

# household_count = 0
# unique_street_count = 0
# addresses_on_street = {}

# for doc in aggregated:
#     household_count += doc['count']
#     unique_street_count += 1
    
#     addresses_on_street[doc['_id']] = doc['count']

# print("total addresses in Bergen:", household_count)
# print("number of streetnames:", unique_street_count)

for doc in user_count:
    print(doc)

### Plan going forward:

- Export list of duplicate addresses in two formats: full data in json/osm, addresses in 2 or 3 columns (3 with postcode)
- Wrangling file: add postcode cleaning

In [ ]:
type({'a':1})

In [ ]:
for key,val in {'type': 'node', '_id': '58ee99f28a51df7ed61f5213',
  'created': {
      'version': '3', 'uid': '114230', 'user': 'danerikk', 'changeset': '6007582', 
      'timestamp': '2010-10-10T22:30:50Z'}, 'id': '358067', 'pos': [60.5314263, 5.2552169]}.items():
    print(key,val)

In [ ]:

features = defaultdict(int)
count = 0

for doc in list(bergen.find()):    

    for k,v in doc.items():
        
        if k == 'node_refs':
            count += 1
        try:
            for key,val in v.items():
#             tmp_set.add(k+'_'+key)
                features[k+':'+key] += 1
#         else:
        except AttributeError:
            if type(v) == list:
                for item in v:
                    if type(item) == dict:
                        for key,val in item.items():
                            features[k+':'+key] += 1
                    else:
                        features[k] += 1
                
            elif type(v) == str:
                features[k] += 1
                if doc['id'] == '616726':
                    print(k,v)
            elif type(v) == dict:
                print('whah')
                print(key)
                print(val)
                break
            elif k != '_id':
                print("SOMETHING FISHY")
                print(k)
                print(v)
                print(type(v))
                print('----')
                break
                
count

In [ ]:
cow_dict = {'a':1,'b':2,'c':3}

1 in cow_dict

In [ ]:
count = 0

for doc in bergen.find({"subject":{'$exists': 1}, "subject.en":{'$exists': 0 } } ):
    count += 1
    
print(count)

In [ ]:
for feature in features.keys():
    if ':' not in feature:
        print(feature,features[feature])

In [ ]:
features

In [ ]:
for key in sorted(features,key=features.get,reverse=True):
    print(key,features[key])

In [ ]:
bergen.find_one({'amenity': {'$exists': True}})

In [ ]:
aggregated = bergen.aggregate([  
        {'$match' :  { 'amenity': {'$exists' : True }} },
        { "$group" : { 
                "_id" : "amenity","count" : { "$sum" : 1} } },
    { "$sort" : { "count" : -1 } }
    ])

# for amenity in aggregated:
#     print(amenity)

In [ ]:
aggregated2 = bergen.aggregate([  
        {'$match' :  { 'leisure': {'$exists' : True }} },
        { "$group" : { 
                "_id" : "$leisure","count" : { "$sum" : 1} } },
    { "$sort" : { "count" : -1 } }
    ])

for doc in aggregated2:
    print(doc)

In [ ]:
for amenity in aggregated:
    for leisure in aggregated2:
        if leisure['_id'] == amenity['_id']:
            print(amenity)
            print(leisure)
            print('----')

In [ ]:
aggregated = bergen.aggregate([  
        {'$match' :  { 'node_refs': {'$exists' : False }} },
        { "$group" : { 
                "_id" : "$amenity","count" : { "$sum" : 1} } },
    { "$sort" : { "count" : -1 } }
    ])

for doc in aggregated:
    print(doc)

In [ ]:
docs_with_feature('name',20)

In [ ]:
for doc in bergen.find( {'name': {'$regex': 'ingo'}}):
    pprint(doc)
    print('----')

In [ ]:
docs_with_feature('gambling')

In [ ]:
for doc in bergen.find( { 'amenity': {'$exists': True}, 'leisure': {'$exists': True} } ):
    pprint(doc)
    print('----')

In [ ]:
from operator import itemgetter

sorted(features.items(), key=itemgetter(1), reverse=True)

In [ ]:
from collections import defaultdict

features = defaultdict(int)

for doc in list(bergen.find( { 'node_refs': {'$exists' : False } } )):    

    for k,v in doc.items():
        
        
        if type(v) == dict:
            for key,val in v.items():
#             tmp_set.add(k+'_'+key)
                features[k+':'+key] += 1
#         else:
        elif type(v) == list:
            for item in v:
                if type(item) == dict:
                    for key,val in item.items():
                        features[k+':'+key] += 1
                else:
                    features[k] += 1

        elif type(v) == str:
            features[k] += 1
            if doc['id'] == '616726':
                print(k,v)

In [ ]:
bergen.find( { 'node_refs': {'$exists' : True } } ).count()

In [ ]:
bergen.find( { 'address.city': {'$exists' : True } } ).count()

In [ ]:
tmp_city = bergen.find( { 'address.city': {'$exists' : True } } )

count = 0

for doc in tmp_city:
    for k,v in doc.items():
        if k == 'address':
            for addr_type,value in v.items():
                if addr_type == 'city':
                    count += 1
                    
                    
count

In [ ]:
for key in sorted(features,key=features.get,reverse=True):
    print(key,features[key])

In [ ]:
for key in sorted(features,key=features.get,reverse=True):
    print(key,features[key])

In [ ]:
# node_ref_count_query = bergen.aggregate( [
#    {
#      '$group': {
#         '_id' : { 'node_refs': '$node_refs'}
#            }
#         },
#     { "$sort" : { "count" : -1 } }
# ] )
       
# for doc in node_ref_count_query:
#        print(doc)
        
# # aggregated = bergen.aggregate([  
# #         {'$match' :  { 'node_refs': {'$exists' : False }} },
# #         { "$group" : { 
# #                 "_id" : "$amenity","count" : { "$sum" : 1} } },
# #     { "$sort" : { "count" : -1 } }
# #     ])

# # for doc in aggregated:
# #     print(doc)

In [11]:
node_refs_cursor = bergen.find( { 'node_refs': {'$exists' : True } } )

node_ref_counts = defaultdict(int)

for doc in node_refs_cursor:
    for node_ref in doc['node_refs']:
        node_ref_counts[node_ref] += 1

In [12]:
count = 0

for key in sorted(node_ref_counts,key=node_ref_counts.get,reverse=True):
    count += 1
    
    if count > 100:
        break
    else:
        print(key,node_ref_counts[key])

29784574 8
659176 7
331537177 7
2958740469 7
3463353550 7
653922 7
388603237 6
3459139122 6
3462350855 6
3463351284 6
1380827404 6
3792014898 6
43522929 6
2028740048 6
3473524850 6
3463351083 6
406569423 6
3464903679 6
21397989 6
3462350892 6
3466223356 6
3466223742 6
3468102863 6
3469622707 6
3469622699 6
197829725 6
2007965551 6
1589792064 6
15992470 6
941663311 6
843232472 6
2958740440 6
3465315221 6
113924307 6
29679011 6
3463353552 6
31092129 6
3463352812 6
21632993 6
29876839 6
3694806525 6
29562543 6
3464903405 6
26082309 6
43523002 6
3465315270 6
3464903021 6
2405180799 6
3437743939 6
14917507 6
2485014527 6
3462996275 6
3464903664 6
3473490762 6
3680075086 6
29943011 6
83075348 5
3473536848 5
3461476470 5
3468151029 5
29922468 5
26082306 5
3464903428 5
3469571846 5
265629308 5
3462014229 5
3469591093 5
29598840 5
3462357486 5
3466225246 5
3463351062 5
125494576 5
3462995841 5
48881406 5
3463353304 5
188991191 5
320778501 5
3695731689 5
289974402 5
17354882 5
746497503 5
346531

In [13]:
bergen.find_one( {'id': '29784574'})

{'_id': ObjectId('58ee99f28a51df7ed61f81e3'),
 'created': {'changeset': '34017753',
  'timestamp': '2015-09-14T10:51:19Z',
  'uid': '715936',
  'user': 'Gazer75',
  'version': '6'},
 'id': '29784574',
 'pos': [60.3864274, 5.3213204],
 'type': 'node'}

In [21]:
node_refs_cursor = bergen.find( { 'node_refs': {'$exists' : True } } )

for doc in node_refs_cursor:
    for node_ref in doc['node_refs']:
        if node_ref == '29784574':
            pprint(doc)
            print('------')

{'_id': ObjectId('58ee9a118a51df7ed628ee29'),
 'created': {'changeset': '14097785',
             'timestamp': '2012-11-30T11:58:27Z',
             'uid': '715936',
             'user': 'Gazer75',
             'version': '2'},
 'highway': 'unclassified',
 'id': '4685644',
 'node_refs': ['29784573', '29784574'],
 'type': 'way'}
------
{'_id': ObjectId('58ee9a128a51df7ed6291a6a'),
 'created': {'changeset': '11749235',
             'timestamp': '2012-05-30T14:35:21Z',
             'uid': '420980',
             'user': 'NorNorth',
             'version': '1'},
 'highway': 'service',
 'id': '165510865',
 'name': 'Menneskerettighetenes plass',
 'node_refs': ['1770495753', '1769659574', '29784577', '29784574', '29784576'],
 'type': 'way'}
------
{'_id': ObjectId('58ee9a128a51df7ed6291e13'),
 'created': {'changeset': '39349421',
             'timestamp': '2016-05-16T11:00:45Z',
             'uid': '1965308',
             'user': 'FredrikLindseth',
             'version': '7'},
 'id': '194290985